**This notebook is for denoising and saving a CT series (After Training and Testing)**

In [ ]:
#Update the variables as their name describes
model_path: str = ''
series_path: str = ''
denoised_series_path: str = ''
sample_steps: int = 1

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
#Run to initialize
import torch
import yaml
import math
import os

from torch.distributed import init_process_group, destroy_process_group

from architectures.UNET.unet import UNET
from architectures.openai.unet import UNetModel

DEEP_MODEL='deep'
OPENAI_MODEL='oai'

def ddp_setup():  
    init_process_group(backend="nccl")
    torch.cuda.set_device(int(os.environ["RANK"])) 

with open("parameters.yaml", 'r') as stream:
    parameters = yaml.safe_load(stream)
ddp_setup()
gpu_id = int(os.environ["RANK"])

if parameters['model_type'] == DEEP_MODEL: 
    model = UNET( img_channels=parameters['img_channels'],  device=gpu_id,groupnorm=parameters['groupnorm'], attention_resolution=parameters['attention_resolutions'], 
                num_heads=parameters['num_heads'], dropout=parameters['dropout'],base_channels=parameters['base_channels'],
                num_res_blocks=parameters['num_res_blocks'],  use_flash_attention=parameters['use_flash_attention'], emb_time_multiplier=parameters['emb_time_multiplier'],
                num_head_channels=parameters['num_head_channels'], use_new_attention_order=parameters['use_new_attention_order'],
                use_scale_shift_norm=parameters['use_scale_shift_norm'],use_conv=parameters['use_conv'], use_conv_up_down =parameters['use_conv_up_down']).to(device=gpu_id)
else:
    model= UNetModel(attention_resolutions=parameters['attention_resolutions'], use_scale_shift_norm=parameters['use_scale_shift_norm'],
                model_channels=parameters['base_channels'],num_head_channels=parameters['num_head_channels'],dropout=parameters['dropout'],
                num_res_blocks=parameters['num_res_blocks'],resblock_updown=True,image_size=parameters['image_size'],in_channels=parameters['img_dimension'],out_channels=parameters['img_dimension'])
model.load_state_dict(torch.load(model_path))
model.to(device=gpu_id)

sigma_min = 0.002
sigma_data = 0.5
sigma_max = 80.0

In [ ]:
#Important methods
def multiple_step_denoising(sample_steps, x): 
    sigmas = get_sigmas_linear_reverse(sample_steps) 
    sample_results = sample(model=model, x=x, ts=sigmas)
    return sample_results

def get_sigmas_linear_reverse(n, sigma_min=0.002, sigma_max=79.999985): 
    sigmas = torch.linspace(sigma_max, sigma_min, n, dtype=torch.float16).to(device=gpu_id)
    return sigmas

def sample(model, x, ts): 
    sigma = ts[0:1]
    sigma = torch.full((x.shape[0],), sigma[0], dtype=x.dtype, device=gpu_id) 
    x = model_forward_wrapper(model, x, sigma)

    for sigma in ts[1:]:
        z = torch.randn_like(x).to(device=gpu_id)
        x = x + math.sqrt(sigma**2 - sigma_min**2) * z
        x = model_forward_wrapper(model, x, torch.tensor([sigma])) 

    return x

def model_forward_wrapper(model, x, sigma):
    x = x.to(device=gpu_id)
    sigma = sigma.to(device=gpu_id)

    c_skip = skip_scaling(sigma)
    c_out = output_scaling(sigma) 
            
    c_skip = pad_dims_like(c_skip, x).to(device=gpu_id)
    c_out = pad_dims_like(c_out, x).to(device=gpu_id)

    return c_skip * x + c_out * model(x, sigma)

def skip_scaling(sigma):
    return sigma_data**2 / ((sigma - sigma_min) ** 2 + sigma_data**2)
    
def output_scaling(sigma):
    return (sigma_data * (sigma - sigma_min)) / (sigma_data**2 + sigma**2) ** 0.5
    
def pad_dims_like(x, other):
    ndim = other.ndim - x.ndim
    return x.view(*x.shape, *((1,) * ndim))


In [ ]:
import pydicom
import numpy as np

series = os.scandir(series_path)

for entry in series:
    name = entry.name
    name = name.split('.')
    if not (entry.is_file() and name[1] == 'dcm'):
        continue
    else:
        ds = pydicom.dcmread(entry.path)

        image = ds.pixel_array
        image = image.astype(np.float32)
        image = torch.from_numpy(image)
        image = torch.unsqueeze(image, 0)
        image = torch.unsqueeze(image, 0) #[1, 1, 512, 512]

        image = image.to(device=gpu_id)

        image = multiple_step_denoising(sample_steps=sample_steps, x=image)

        image = torch.squeeze(image)
        image = image.numpy(force=True)

        image = (image - np.min(image))
        image = image/np.max(image)
        image = (image * 4095).astype(np.uint8)

        ds.PixelData = image.tobytes()
        ds.save_as(f'{name[0]}-denoised')

series.close()

In [ ]:
destroy_process_group()